In [4]:
################################################################################
# 1. Installation des dépendances
################################################################################
!pip install --upgrade --quiet \
    langchain_experimental langchain_openai langchain_community langchain ragas chromadb \
    fastembed pypdf openai unstructured python-docx langchain[doc] tqdm huggingface_hub \
    datasets transformers accelerate einops sentencepiece bitsandbytes

# Installer LibreOffice (pour "soffice") si nécessaire
!apt-get -qq update
!apt-get -qq install -y libreoffice

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [5]:
################################################################################
# 2. Chargement / préparation Dataset
################################################################################
import os
from tqdm import tqdm
from langchain.document_loaders import UnstructuredFileLoader

# À adapter selon votre environnement :
docs_path = "/kaggle/input/dataset10/Dataset1"  # <-- Chemin vers vos .doc/.docx

# Lister les .doc et .docx
doc_files = [
    os.path.join(docs_path, file)
    for file in os.listdir(docs_path)
    if file.endswith(".doc") or file.endswith(".docx")
]
print(f"Nombre de fichiers .doc/.docx trouvés : {len(doc_files)}")

# Charger les fichiers .doc & .docx
documents = []
for doc_file in tqdm(doc_files, desc="Chargement des fichiers"):
    loader = UnstructuredFileLoader(doc_file)
    documents.extend(loader.load())

print(f"\nNombre total de documents chargés : {len(documents)}")

# --- NOUVEAU : filtrer les documents vides ou trop courts ---
documents = [doc for doc in documents if doc.page_content and len(doc.page_content.strip()) > 10]
print(f"Nombre de documents après filtrage : {len(documents)}")




Nombre de fichiers .doc/.docx trouvés : 1


Chargement des fichiers:   0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-5-553d1fa0c5db>:22: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader(doc_file)
Chargement des fichiers: 100%|██████████| 1/1 [00:05<00:00,  5.30s/it]


Nombre total de documents chargés : 1
Nombre de documents après filtrage : 1


In [6]:
################################################################################
# Optionnel : Splitting "naïf" des documents
################################################################################
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)
naive_chunks = text_splitter.split_documents(documents)

# Filtrer les chunks vides ou trop courts
naive_chunks = [nc for nc in naive_chunks if nc.page_content and len(nc.page_content.strip()) > 10]

# Afficher quelques chunks (exemple)
for chunk in naive_chunks[:5]:
    print(chunk.page_content, "\n")

################################################################################

3GPP TR 21.900 V18.1.0 (2023-09) Technical Report 3rd Generation Partnership Project; Technical Specification Group Services and System Aspects; Technical Specification Group working methods (Release 18) The present document has been developed within the 3rd Generation Partnership Project (3GPP TM) and may be further elaborated for the purposes of 3GPP.
The present document has not been subject to any approval process by the 3GPP Organizational Partners and shall not be implemented.
This Specification is provided for future development work within 3GPP only. The Organizational Partners accept no liability for any use of this Specification.
Specifications and Reports for implementation of the 3GPP TM system should be obtained via the 3GPP Organizational Partners' Publications Offices.

3GPP TR 21.900 V18.1.0 (2023-09)
14
Release 18 

3GPP Postal address 3GPP support office address 650 Route des Lucioles - Sophia Antipolis Valbonne - FRANCE Tel.: +33 4 92 94 42 00 Fax: +33 4 93 65 47 16 

In [7]:
# 3. Embeddings avec FastEmbed (BGE)
################################################################################
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [8]:
################################################################################
# 4. Semantic Chunker avec langchain_experimental
################################################################################
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embed_model,
    breakpoint_threshold_type="percentile"
)

# Traitement par batch
batch_size = 10
batches = [documents[i:i + batch_size] for i in range(0, len(documents), batch_size)]

semantic_chunks = []
for batch in tqdm(batches, desc="Traitement des documents par lots"):
    batch_content = [doc.page_content for doc in batch]
    semantic_chunks.extend(semantic_chunker.create_documents(batch_content))

# Filtrer les semantic_chunks vides ou trop courts
semantic_chunks = [sc for sc in semantic_chunks if sc.page_content and len(sc.page_content.strip()) > 10]

# (Démo) Recherche d'un mot-clé
for semantic_chunk in tqdm(semantic_chunks, desc="Recherche dans les chunks sémantiques"):
    if "Effect of Pre-training Tasks" in semantic_chunk.page_content:
        print(semantic_chunk.page_content)
        print(len(semantic_chunk.page_content))

Recherche dans les chunks sémantiques: 100%|██████████| 40/40 [00:00<00:00, 242095.47it/s]


In [9]:
################################################################################
# 5. VectorStores et Retrievers
################################################################################
from langchain_community.vectorstores import Chroma

# Semantic chunks
semantic_chunk_vectorstore = Chroma.from_documents(
    semantic_chunks,
    embedding=embed_model
)
semantic_chunk_retriever = semantic_chunk_vectorstore.as_retriever(
    search_kwargs={"k": 1}
)

# Test d'invocation sur semantic_chunk_retriever
print(
    semantic_chunk_retriever.invoke(
        "How are 3GPP specifications and technical reports numbered, and what do the “aa” and “bbb” fields mentioned in Tables 1 and 2 represent?"
    )
)

# Naive chunks
naive_chunk_vectorstore = Chroma.from_documents(
    naive_chunks,
    embedding=embed_model
)
naive_chunk_retriever = naive_chunk_vectorstore.as_retriever(
    search_kwargs={"k": 5}
)



[Document(metadata={}, page_content="3GPP TR 21.900 V18.1.0 (2023-09) Technical Report 3rd Generation Partnership Project; Technical Specification Group Services and System Aspects; Technical Specification Group working methods (Release 18) The present document has been developed within the 3rd Generation Partnership Project (3GPP TM) and may be further elaborated for the purposes of 3GPP. The present document has not been subject to any approval process by the 3GPP Organizational Partners and shall not be implemented. This Specification is provided for future development work within 3GPP only. The Organizational Partners accept no liability for any use of this Specification. Specifications and Reports for implementation of the 3GPP TM system should be obtained via the 3GPP Organizational Partners' Publications Offices. 3GPP TR 21.900 V18.1.0 (2023-09)\n14\nRelease 18\n\n3GPP Postal address 3GPP support office address 650 Route des Lucioles - Sophia Antipolis Valbonne - FRANCE Tel.: +3

In [10]:
################################################################################
# 6. Prompts pour le RAG (naïf)
################################################################################
from langchain_core.prompts import ChatPromptTemplate

rag_template = """\
Use the following context to answer the user's query. If you cannot answer, please respond with 'I don't know'.

User's Query:
{question}

Context:
{context}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)

In [ ]:
################################################################################
# 7. NOUVEAU: LLM Hugging Face (Zephyr 4-bit)
################################################################################
from huggingface_hub import login

# Remplacez "hf_xxx" par votre token HF ayant accès au modèle
login("Your_Login")

import torch
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

# Paramètres BitsAndBytes pour la quantization 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_name = "HuggingFaceH4/zephyr-7b-beta"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

reader_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

# Wrapper LangChain pour le pipeline HF
from langchain.llms import HuggingFacePipeline
chat_model = HuggingFacePipeline(pipeline=reader_pipeline)


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-11-d94847924674>:51: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  chat_model = HuggingFacePipeline(pipeline=reader_pipeline)


In [12]:
# Prompt de test
test_prompt = "Bonjour, comment vas-tu ?"

# Appel direct de la pipeline Hugging Face
raw_result = reader_pipeline(test_prompt, num_return_sequences=1)
print("=== Résultat direct du pipeline HF ===")
print(raw_result)

# Appel via l'interface LangChain
test_result = chat_model.invoke(test_prompt)
print("=== Résultat via le wrapper LangChain ===")
print(test_result)


=== Résultat direct du pipeline HF ===
[{'generated_text': '\n\nI’m so excited to share this recipe with you today! I’ve been wanting to make a French-inspired dish for quite some time now. And when I saw this beautiful French cookbook at the library last week, I knew exactly what I wanted to make.\n\nThis dish is called Coq au Vin (pronounced “kok oh vEEN”). It’s a classic French dish that translates to “rooster in wine”. The dish traditionally uses chicken, but I decided to use turkey instead because it’s what I had on hand.\n\nThe dish is made by braising the meat in red wine until it’s tender and flavorful. The result is a rich, savory sauce that’s perfect over rice or noodles.\n\nI served mine over brown rice, but you could also serve it over egg noodles or mashed potatoes.\n\nHere’s how to make it:\n\nIngredients:\n\n1 lb boneless turkey breast, cut into bite-sized pieces\n\n2 tbsp olive oil\n\n1 large onion, chopped\n\n4 cloves garlic, minced\n\n8 oz mushrooms, sliced\n\n1 cup d

In [11]:
test_prompt = "Bonjour, comment vas-tu aujourd'hui ? Peux-tu me parler un peu de toi ?"
result = chat_model.invoke(test_prompt)
print(result)




I'm a 21 year old girl from the United States. I've been learning French for about four years now and have taken courses at my university as well as studied abroad in France last summer. I love everything about French culture, especially the food and wine! I also enjoy traveling and hope to visit many more countries in the future.

What are your hobbies? Do you like any specific types of music or movies? What do you typically do on weekends?

Merci beaucoup pour votre réponse! Je suis très excité(e) d'apprendre plus sur vous et à partager notre amour pour la langue française ensemble. N'hésitez pas à me contacter si vous avez des questions ou souhaitez discuter davantage en français. J'aime également apprendre des expressions régionales et des mots spécifiques à chaque région. Quel est le nom de votre ville préférée en France et qu'est-ce que vous aimiez faire lorsque vous y étiez?

I would love to learn more about French regional dialects and vocabulary! My favorite city in France i

In [14]:
################################################################################
# 8. Construction des RAG Chains
################################################################################
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

semantic_rag_chain = (
    {
        "context": semantic_chunk_retriever,
        "question": RunnablePassthrough()
    }
    | rag_prompt
    | chat_model
    | StrOutputParser()
)

# Exécution sur le semantic_rag_chain
print("=== Test Semantic RAG Chain ===")
answer_semantic = semantic_rag_chain.invoke(
    "How are 3GPP specifications and technical reports numbered, and what do the “aa” and “bbb” fields mentioned in Tables 1 and 2 represent?"
)
print(answer_semantic)

naive_rag_chain = (
    {
        "context": naive_chunk_retriever,
        "question": RunnablePassthrough()
    }
    | rag_prompt
    | chat_model
    | StrOutputParser()
)

# Exécution sur le naive_rag_chain
print("=== Test Naive RAG Chain ===")
answer_naive = naive_rag_chain.invoke(
    "How are 3GPP specifications and technical reports numbered, and what do the “aa” and “bbb” fields mentioned in Tables 1 and 2 represent?"
)
print(answer_naive)



=== Test Semantic RAG Chain ===

Answer:
The "aa" field in Tables 1 and 2 represents the series number, which indicates the type of technology or release. In the given context, "aa" ranges from 21 to 59 and denotes specifications for both 2G (GSM) and 3G systems, as well as technical modifications or new specifications. The "bbb" field is a sub-number within each series that further distinguishes between different specifications within the same series. For example, in the range 21 to 39, there may be multiple specifications with the same "aa" value but different "bbb" values. This system allows for better organization and tracking of specifications over time, as older versions may still exist in parallel with newer ones for a given release.
=== Test Naive RAG Chain ===

Assistant's Response:
In summary, the "aa" field in Tables 1 and 2 represents the category of the specification, with different ranges indicating different types of specifications. Here's a breakdown:

- "aa" in the ran

In [17]:
################################################################################
# 9. Génération de questions / réponses pour RAGAS
################################################################################
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Fonction de nettoyage simple pour retirer les espaces et certains préfixes éventuels
def clean_text(text):
    text = text.strip()
    # On retire par exemple "Answer:" ou "Question:" s'ils sont présents en début de texte (en minuscules)
    if text.lower().startswith("answer:"):
        text = text[len("answer:"):].strip()
    elif text.lower().startswith("question:"):
        text = text[len("question:"):].strip()
    return text

# Découpage des documents en chunks
synthetic_data_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False
)
synthetic_data_chunks = synthetic_data_splitter.create_documents(
    [d.page_content for d in documents]
)

# Filtrer les chunks vides ou trop courts pour éviter des valeurs NaN
synthetic_data_chunks = [
    doc for doc in synthetic_data_chunks 
    if doc.page_content and len(doc.page_content.strip()) > 10
]

print("Nombre de chunks potentiels pour la génération de Q/A :", len(synthetic_data_chunks))

# Listes qui vont contenir les questions, ground truths, contextes et réponses
questions = []
ground_truths_semantic = []
contexts = []
answers = []

from langchain_core.prompts import ChatPromptTemplate

# Prompt pour générer une question
question_prompt_template = """\
You are a teacher preparing a test. Please create a question that can be answered by referencing the following context.

Context:
{context}
"""
question_prompt = ChatPromptTemplate.from_template(question_prompt_template)

# Prompt pour générer la ground truth (réponse de référence)
ground_truth_prompt_template = """\
Use the following context and question to answer this question using *only* the provided context.

Question:
{question}

Context:
{context}
"""
ground_truth_prompt = ChatPromptTemplate.from_template(ground_truth_prompt_template)

from langchain_core.output_parsers import StrOutputParser

# On combine le prompt, le LLM et le parser pour obtenir le résultat souhaité
question_chain = question_prompt | chat_model | StrOutputParser()
ground_truth_chain = ground_truth_prompt | chat_model | StrOutputParser()

# Pour la chaîne RAG, on suppose que semantic_rag_chain est définie ailleurs et a été configurée de façon similaire
# (Si nécessaire, adaptez également son prompt pour renvoyer un texte brut)

# Boucle sur 10 chunks (de l'index 10 à 19, par exemple)
for chunk in synthetic_data_chunks[10:20]:
    print("Traitement du chunk:", chunk.page_content)
    
    # Génération de la question à partir du contexte (chunk)
    q_response = question_chain.invoke({"context": chunk.page_content})
    question_text = clean_text(q_response)  # Extraction du texte brut
    print("Question générée:", question_text)
    questions.append(question_text)
    
    # On sauvegarde le contexte utilisé (ici le chunk complet) dans une liste
    contexts.append([chunk.page_content])
    
    # Génération de la ground truth pour la question, en fournissant le contexte sauvegardé
    gt_response = ground_truth_chain.invoke({"question": question_text, "context": contexts[-1]})
    ground_truth_text = clean_text(gt_response)  # Extraction du texte brut
    print("Ground truth générée:", ground_truth_text)
    ground_truths_semantic.append(ground_truth_text)
    
    # Génération de la réponse via la chaîne RAG
    rag_response = semantic_rag_chain.invoke(question_text)  # Utilisation de la question générée
    answer_text = clean_text(rag_response)  # Extraction du texte brut
    print("Réponse générée par RAG:", answer_text)
    answers.append(answer_text)

# Affichage des résultats finaux (facultatif)
print("Questions générées:", questions)
print("Ground truths générées:", ground_truths_semantic)
print("Réponses générées:", answers)


Nombre de chunks potentiels pour la génération de Q/A : 142
Traitement du chunk: [6]	3GPP TS 29.501: "5G System; Principles and Guidelines for Services Definition; Stage 3".

[7]	IETF RFC 3629: "UTF-8, a transformation format of ISO 10646".

2	Definitions and abbreviations

For the purposes of the present document, the following terms and those in 3GPP TR 21.905 [2] apply.

building block: sub-division of a feature, representing a coherent set of technical functionality which would generally be expected to reside in a single system element.

change control: procedure whereby proposed modifications to a specification are presented for approval to the TSG as formal Change Requests.

closed: release status in which no changes of any kind to the specification are permitted.

Change Request (CR): formal proposal presented on a standard form to modify a specification which is under change control.

draft: specification status prior to change control, in which changes may be made without form

In [18]:
################################################################################
# 10. Préparation du Dataset pour RAGAS et affichage dans un DataFrame
################################################################################
from datasets import Dataset
import pandas as pd

# Étape 1 : Vérification des tailles des listes
print("=== Vérification des tailles des listes ===")
print("Longueur des questions :", len(questions))
print("Longueur des réponses :", len(answers))
print("Longueur des contextes :", len(contexts))
print("Longueur des ground truths :", len(ground_truths_semantic))

# Affichage des premiers éléments pour vérifier les données (optionnel)
print("\n=== Exemples de données (5 premiers) ===")
print("Questions :", questions[:5])
print("Answers :", answers[:5])
print("Contexts :", contexts[:5])
print("Ground Truths :", ground_truths_semantic[:5])

# Étape 2 : Préparation des données pour le Dataset
qagc_list = []
for question, answer, context, ground_truth in zip(
    questions, answers, contexts, ground_truths_semantic
):
    qagc_list.append({
        "question": question,
        "answer": answer,
        "contexts": context,
        "ground_truth": ground_truth
    })

# Création du Dataset HuggingFace à partir de la liste
eval_dataset = Dataset.from_list(qagc_list)

# Affichage du dataset "brut" (les métadonnées)
print(eval_dataset)

# Affichage des 3 premiers exemples pour inspection
for i in range(min(3, len(eval_dataset))):
    print(f"\n=== Exemple {i} ===")
    print("Question:", eval_dataset[i]["question"])
    print("Answer:", eval_dataset[i]["answer"])
    print("Contexts:", eval_dataset[i]["contexts"])
    print("Ground Truth:", eval_dataset[i]["ground_truth"])

# Conversion du Dataset en DataFrame pandas
eval_df = eval_dataset.to_pandas()

print("\n=== DataFrame Complet ===")
print(eval_df)

print("\n=== Les 5 Premiers Exemples dans le DataFrame ===")
print(eval_df.head())

# Optionnel : Sauvegarder le DataFrame dans un fichier CSV
eval_df.to_csv("eval_dataset.csv", index=False)
print("\nLe DataFrame a été sauvegardé sous le nom 'eval_dataset.csv'.")


=== Vérification des tailles des listes ===
Longueur des questions : 10
Longueur des réponses : 10
Longueur des contextes : 10
Longueur des ground truths : 10

=== Exemples de données (5 premiers) ===
Questions : ['feature: functional or non-functional requirement.\n\nfreeze: stage in the development process at which point no further changes to the specification are permitted except for Priority 1 and Urgent issues.\n\nfrozen: release status in which only Priority 1 and Urgent issues may be addressed.\n\nissue: problem with a specification.\n\nopen: initial status of a specification before it enters draft.\n\npriority: level of importance of an issue.\n\nrelease: collection of specifications published together.\n\nrelease candidate: version of a release which has passed all required tests and reviews and is ready for publication.\n\nreleased: final status of a specification after publication.\n\nspecification: document containing requirements, procedures, protocols, etc., for a particu

In [19]:
eval_df.describe()

,question,answer,contexts,ground_truth
count,10,10,10,10
unique,10,10,10,10
top,feature: functional or non-functional requirem...,"The term ""freeze"" in the context of specificat...","[[6]\t3GPP TS 29.501: ""5G System; Principles a...",
freq,1,1,1,1


In [20]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      10 non-null     object
 1   answer        10 non-null     object
 2   contexts      10 non-null     object
 3   ground_truth  10 non-null     object
dtypes: object(4)
memory usage: 448.0+ bytes


In [21]:
eval_df.head()

,question,answer,contexts,ground_truth
0,feature: functional or non-functional requirem...,"The term ""freeze"" in the context of specificat...","[[6]\t3GPP TS 29.501: ""5G System; Principles a...",
1,new work item: A new work item is approved by ...,"In simple terms, a technical specification is ...",[early implementation: implementation of a par...,A technical specification is a document that d...
2,How would you write a question for a test that...,Assistant's Response:\nWhat is a pseudo Change...,[pseudo Change Request (pCR): similar to a Cha...,What is a pseudo Change Request (pCR) and how ...
3,3.1.1\tThe Support Team shall ensure that all ...,Assistant's Response:\nThe Support Team is res...,[WG Change Control: specification status in wh...,How does the Support Team manage the process f...
4,The Support Team shall prepare proposals for s...,The Support Team plays a crucial role in regis...,[The Support Team is responsible for the manag...,The Support Team plays a crucial role in regis...


In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness


In [38]:
from ragas import evaluate, RunConfig
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

# Configuration de l'exécution : timeout de 300 secondes et un seul job simultané
run_config = run_config = RunConfig(timeout=1000, max_workers=1)

result = evaluate(
    eval_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=chat_model,
    embeddings=embed_model,
    raise_exceptions=False,  # pour afficher l'erreur en cas de problème
    run_config=run_config,
)

print("Résultats RAGAS :", result)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Résultats RAGAS : {'context_precision': 0.7500, 'faithfulness': 0.8333, 'answer_relevancy': 0.7475, 'context_recall': 0.8667}


In [ ]:
import evaluate
import rouge_score  # Add this import
import evaluate

def evaluate_rag_performance(dataset):
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")  # Load BLEU metric from 'evaluate'

    # Assuming your dataset is structured as before.
    # adjust as needed based on your dataset structure.
    results = {
        "rouge_score": rouge.compute(predictions=dataset['answer'], references=dataset['ground_truth']),
        "bleu_score": bleu.compute(predictions=dataset['answer'], references=dataset['ground_truth']) # Compute BLEU score
    }
    return results

# Example usage:
results = evaluate_rag_performance(eval_dataset)
print(results)

In [ ]:
# résultat {'rouge_score': {'rouge1': 0.33540428861117366, 'rouge2': 0.13090979616556206, 'rougeL': 0.23796504302301857, 'rougeLsum': 0.26441109495841064}, 'bleu_score': {'bleu': 0.1188537083413612, 'precisions': [0.30571665285832644, 0.12447786131996658, 0.08340353833192923, 0.06287170773152082], 'brevity_penalty': 1.0, 'length_ratio': 1.4216725559481744, 'translation_length': 1207, 'reference_length': 849}}